# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/yolov8_tracking.git  # clone repo
%pip install -qr requirements.txt  # install dependencies
%pip install thop                  # install dependencies
%pip install sentry_sdk
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
from google.colab import files
upload = files.upload()
dataset =  list(upload.keys())[0]

In [ ]:
#!unzip /content/datasets/test_mouse.zip -d /content/datasets/
!unzip $dataset

# Train a new model based on the custom new dataset

In [ ]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n-seg.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
model.train(data="/content/datasets/test_mouse/data.yaml", epochs=100)

# Download data

Get test video from repo and extract the first 2 seconds of it 

In [ ]:
%cd /content/yolov8_tracking

In [ ]:

# upload your video or get the test video from the repo
!wget -nc https://github.com/mikel-brostrom/yolov8_tracking/releases/download/v.2.0/test.avi
# extract 3 seconds worth of video frames of it
!yes | ffmpeg -ss 00:00:00 -i test.avi -t 00:00:01 -vf fps=30 out.avi

## Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935. 

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time

In [ ]:
!python track.py --yolo-weights /content/yolov8_tracking/runs/track/detect/train3/weights/last.pt --tracking-method ocsort --reid-weights osnet_x0_25_msmt17.pt --source /mouse.mp4 --save-vid

In [ ]:
!pip install pytube

In [ ]:
from pytube import YouTube
import os

def downloadYouTube(videourl, path):

    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    if not os.path.exists(path):
        os.makedirs(path)
    yt.download(path)

downloadYouTube('https://youtu.be/58z1OuCd6m8', '/content')

In [ ]:
!python track.py --yolo-weights yolov8n-seg.pt --tracking-method ocsort --reid-weights osnet_x0_25_msmt17.pt --source '/content/bird.mp4' --save-vid


# Show results

https://stackoverflow.com/questions/60977179/how-to-play-avi-file-in-google-colab

https://stackoverflow.com/questions/57377185/how-play-mp4-video-in-google-colab

Compress the video file to a format supported by Google Colab (mpeg4 (native) -> h264 (libx264))

In [ ]:
!ffmpeg -i /content/yolov8_tracking/runs/track/exp11/bird.mp4 -vf fps=30 -vcodec libx264 output.mp4

Get the file content into data_url

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

Display it with HTML

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Reference: https://github.com/mikel-brostrom/yolov8_tracking